In [1]:






def MollweideFunc (latitude, epsilon):
    MollweideFunc = lambda alpha : 2 * alpha + numpy.sin(2 * alpha) - numpy.pi * numpy.sin ( latitude )
        
    alpha = fsolve(MollweideFunc, latitude, xtol=epsilon)
        
    return alpha[0]
    
def GeographicToMollweide (geographicPoints):
    epsilon = 1e-6
        
    MollweidePoints = numpy.array(numpy.zeros(geographicPoints.shape))
        
    alpha = [MollweideFunc(lat, epsilon) if not numpy.isclose(numpy.absolute(lat), numpy.pi/2, rtol=epsilon, atol=0.0, equal_nan=False) else lat for lat in geographicPoints[:, 1]]
    MollweidePoints[:, 0] = (2 * numpy.sqrt(2) / numpy.pi) * ( geographicPoints[:, 0] - numpy.pi ) * numpy.cos(alpha)
    MollweidePoints[:, 1] = numpy.sqrt(2) * numpy.sin(alpha)
        
    return MollweidePoints
    
def tangent_geographic_to_Cartesian (points, dpoints):
    initial_vectors = numpy.array(points, dtype='float128', copy=True, order='K', subok=False, ndmin=0)
    dpoints = numpy.array(dpoints, dtype='float128', copy=True, order='K', subok=False, ndmin=0)
    
    end_vectors = initial_vectors + dpoints

    initial_vectors = geographic_to_Cartesian (initial_vectors)
    end_vectors = geographic_to_Cartesian (end_vectors)

    t = numpy.reciprocal ( numpy.einsum ( 'ij,ij->i' , initial_vectors , end_vectors ) )
    tangent_vectors = numpy.einsum ( 'ij,i->ij' , end_vectors , t ) - initial_vectors
    
    return tangent_vectors

def tangent_geographic_to_Cartesian2 (points, dpoints):
    tangent_vector = numpy.zeros ( ( len(points) , 3 ) , dtype = float)
    
    theta = numpy.pi / 2 - points[... , 1]
    phi = points[... , 0]
    
    dtheta = - dpoints[... , 1]
    dphi = dpoints[... , 0]
    
    tangent_vector[...,0] = numpy.cos (theta) * numpy.cos (phi) * dtheta - numpy.sin (theta) * numpy.sin (phi) * dphi
    tangent_vector[...,1] = numpy.cos (theta) * numpy.sin (phi) * dtheta + numpy.sin (theta) * numpy.cos (phi) * dphi
    tangent_vector[...,2] = - numpy.sin (theta) * dtheta
    
    return tangent_vector

def uniform_points_on_sphere_from_data (data , N):
    starting_point_ind = random.SystemRandom().randint (1 , len(data))

    points_from_data = numpy.array([starting_point_ind])
    pairwise_distances_to_data = data.dot(data[starting_point_ind].transpose())

    for i in range(1, N):
        k = pairwise_distances_to_data.argmin()

        points_from_data = numpy.append(points_from_data, k)

        new_distances_to_data = numpy.array(data.dot(data[k].transpose()))

        pairwise_distances_to_data = numpy.maximum (pairwise_distances_to_data, new_distances_to_data)
        
    return (points_from_data)

def switch_hemispheres_geographic_coords (points):
    new_points = points.copy()
    
    new_points[...,0] = numpy.where ( points[...,0] < numpy.pi , points[...,0] + numpy.pi , points[...,0] - numpy.pi )
    
    return new_points

# def simple_input_func (path):
#     points = []
    
#     with open(path) as csvfile:
#         reader = csv.reader ( csvfile )
#         next ( reader )
        
#         for row in reader:
#             ra = numpy.deg2rad ( float ( row[5] ) )
#             dec = numpy.deg2rad ( float ( row[7] ) )
            
#             points.append ( [ ra , dec ] )
    
#     points = numpy.array ( points )
#     return points

# def simple_input_func_pm (path):
#     points = []
    
#     with open(path) as csvfile:
#         reader = csv.reader ( csvfile )
#         next ( reader )
        
#         for row in reader:
#             if row[12] == "NULL":
#                 ra = numpy.nan
#             else:
#                 print(row[12])
#                 ra = numpy.deg2rad ( float ( row[12] ) )
                
#             if row[14] == "NULL":
#                 dec = numpy.nan
#             else:
#                 dec = numpy.deg2rad ( float ( row[14] ) )
            
#             points.append ( [ ra , dec ] )
    
#     points = numpy.array ( points )
#     return points

# Data import

In [1]:
import pandas
import numpy

class AstrometricDataframe:
    def __init__(self): 
        self.positions = numpy.array ([])
        self.positions_coord_system = ""

        self.positions_err = numpy.array ([])

        self.proper_motions = numpy.array ([])

        self.proper_motions_err = numpy.array ([])
        
        self.proper_motions_err_corr = numpy.array ([])
    

def import_Gaia_data (path_to_Gaia_data):
    dataset = pandas.read_csv(path_to_Gaia_data,
                              sep=',',
                              delimiter=None,
                              header='infer',
                              names=None,
                              index_col=None,
                              usecols=None,
                              squeeze=False,
                              prefix=None,
                              mangle_dupe_cols=True,
                              dtype=None,
                              engine='python',
                              converters=None,
                              true_values=None,
                              false_values=None,
                              skipinitialspace=False,
                              skiprows=None,
                              skipfooter=0,
                              nrows=None,
                              na_values=None,
                              keep_default_na=True,
                              na_filter=True,
                              verbose=False,
                              skip_blank_lines=True,
                              parse_dates=False,
                              infer_datetime_format=False,
                              keep_date_col=False,
                              date_parser=None,
                              dayfirst=False,
                              iterator=False,
                              chunksize=None,
                              compression=None,
                              thousands=None,
                              decimal=b'.',
                              lineterminator=None,
                              quotechar='"',
                              quoting=0,
                              doublequote=True,
                              escapechar=None,
                              comment=None,
                              encoding=None,
                              dialect=None,
                              error_bad_lines=True,
                              warn_bad_lines=True,
                              delim_whitespace=False,
                              low_memory=True,
                              memory_map=False,
                              float_precision=None)
    
    dropna_columns = ['ra',
                     'dec',
                     'ra_error',
                     'dec_error',
                     'pmra',
                     'pmdec',
                     'pmra_error',
                     'pmdec_error',
                     'pmra_pmdec_corr']

    dataset.dropna(axis=0,
                   how='any',
                   thresh=None,
                   subset=dropna_columns,
                   inplace=True)
    
    new_dataframe = AstrometricDataframe()
    
    new_dataframe.positions = dataset.as_matrix ( columns = [ 'ra' , 'dec' ] )
    new_dataframe.positions_coord_system = "Geographic"
    
    new_dataframe.positions_err = dataset.as_matrix ( columns = [ 'ra_error' , 'dec_error' ] )
    
    new_dataframe.proper_motions = dataset.as_matrix ( columns = [ 'pmra' , 'pmdec' ] )
    
    new_dataframe.proper_motions_err = dataset.as_matrix ( columns = [ 'pmra_error' , 'pmdec_error' ] )
    
    new_dataframe.proper_motions_err_corr = dataset.as_matrix ( columns = [ 'pmra_pmdec_corr' ] )
    
    return new_dataframe
    

In [2]:
data3 = import_Gaia_data("data/type2.csv")

In [44]:
data3.positions = data3.positions[:100]
data3.positions_err = data3.positions_err[:100]
data3.proper_motions = data3.proper_motions[:100]
data3.proper_motions_err = data3.proper_motions_err[:100]
data3.proper_motions_err_corr = data3.proper_motions_err_corr[:100]

# Data conversion and coordinates change

In [45]:
import csv
import numpy
import random
import time

from scipy.optimize import fsolve

def deg_to_rad(degree_vals):
    return numpy.deg2rad (degree_vals)

In [46]:
# Change positions from deg to rad

data3.positions = deg_to_rad (data3.positions)

In [47]:
# Change proper motions from mas/yr to rad/s

data3.proper_motions = data3.proper_motions * 1.5362818500441604e-16
data3.proper_motions_err = data3.proper_motions_err * 1.5362818500441604e-16

In [6]:
# Change proper motions from (ra,dec) to Cartesian coordinates

#data3.proper_motions = tangent_geographic_to_Cartesian2 (data3.positions , data3.proper_motions)
#data3.proper_motions_err = tangent_geographic_to_Cartesian2 (data3.positions , data3.proper_motions_err)

In [7]:
# Change positions from (ra,dec) to Cartesian

#data3.positions = geographic_to_Cartesian (data3.positions)

# Generating random VSH coefficients

In [48]:
import random
from VectorSphericalHarmonicsVectorized import VectorSphericalHarmonicE, VectorSphericalHarmonicB

def random_aQlm_coeffs ( lmax , lower_bound , upper_bound ):
    negative_coeffs = [ [ random.uniform ( lower_bound , upper_bound ) + 1j * random.uniform ( lower_bound , upper_bound ) for m in range ( -l , 0 ) ] for l in range ( 1 , lmax + 1 ) ]
    
    coeffs = [ [ negative_coeffs[ l-1 ][ m+l ] if m < 0
                 else random.uniform ( lower_bound , upper_bound ) + 1j * 0.0 if m == 0
                 else (-1) ** m * numpy.conj ( negative_coeffs[ l-1 ][ m-l ] )
                 for m in range ( -l , l+1 ) ] for l in range ( 1 , lmax + 1 ) ]
    
    return coeffs

def random_vsh_coeffs ( lmax , lower_bound , upper_bound):
    vsh_E_coeffs = random_aQlm_coeffs ( lmax , lower_bound , upper_bound )
    vsh_B_coeffs = random_aQlm_coeffs ( lmax , lower_bound , upper_bound )
        
    return vsh_E_coeffs , vsh_B_coeffs


In [49]:
vsh_E_coeffs , vsh_B_coeffs = random_vsh_coeffs ( 2 , -1.0e-15 , 1.0e-15)

# Generate data model

In [50]:
def geographic_to_Cartesian (points):
    if len ( points.shape ) == 1:
        nrows = 1
    else:
        nrows = points.shape[0]
        
    new_points = numpy.zeros ( ( len(points) , 3 ))
    
    theta = numpy.pi / 2 - points[... , 1]
    phi = points[... , 0]
    
    new_points[...,0] = numpy.sin ( theta ) * numpy.cos ( phi )
    new_points[...,1] = numpy.sin ( theta ) * numpy.sin ( phi )
    new_points[...,2] = numpy.cos ( theta )
    
    if len ( points.shape ) == 1:
        return new_points[0]
    else:
        return new_points

def tangent_Cartesian_to_geographic (points , dpoints):
    if points.ndim == 1:
        tangent_vector = numpy.zeros ( ( 2 ) , dtype = float)
    else:
        tangent_vector = numpy.zeros ( ( len(points) , 2 ) , dtype = float)
    
    x = points[... , 0]
    y = points[... , 1]
    z = points[... , 2]
    
    dx = dpoints[... , 0]
    dy = dpoints[... , 1]
    dz = dpoints[... , 2]
    
    tangent_vector[... , 0] = dz / ( numpy.sqrt ( 1 - z ** 2 ) )
    tangent_vector[... , 1] = ( x * dy - y * dx ) / ( x ** 2 + y ** 2 )
    
    return tangent_vector

def generate_model ( vsh_E_coeffs , vsh_B_coeffs , positions ):
    lmax = min ( len( vsh_E_coeffs ) , len( vsh_B_coeffs ) )

    positions_Cartesian = geographic_to_Cartesian ( positions )
    
    v_E = numpy.sum ( [ numpy.sum ( [ vsh_E_coeffs[ l-1 ][ m+l ] * VectorSphericalHarmonicE ( l , m , positions_Cartesian ) for m in range ( -l , l+1 ) ] , axis = 0 ) for l in range ( 1 , lmax + 1 ) ] , axis = 0 )
    
    v_B = numpy.sum ( [ numpy.sum ( [ vsh_B_coeffs[ l-1 ][ m+l ] * VectorSphericalHarmonicB ( l , m , positions_Cartesian ) for m in range ( -l , l+1 ) ] , axis = 0 ) for l in range ( 1 , lmax + 1 ) ] , axis = 0 )
    
    numpy.testing.assert_almost_equal(numpy.imag(v_E).sum(), 0.)
    numpy.testing.assert_almost_equal(numpy.imag(v_B).sum(), 0.)
    
    v_Q = numpy.real ( numpy.add ( numpy.array ( v_E ) , numpy.array ( v_B ) ) )
        
    return tangent_Cartesian_to_geographic ( positions_Cartesian , v_Q )


In [51]:
model = generate_model ( vsh_E_coeffs , vsh_B_coeffs , data3.positions )

# Compute the log-likelihood

In [52]:
def covariant_matrix ( errors , corr ):
    covariant_matrix = numpy.einsum ( '...i,...j->...ij' , errors , errors )
    
    covariant_matrix[...,0,1] = covariant_matrix[...,1,0] = numpy.multiply ( covariant_matrix[...,1,0] , corr.flatten() )
    return covariant_matrix

def R_values ( pm , pm_err , pm_err_corr , model ):
    covariant_matrices = covariant_matrix ( pm_err , pm_err_corr )
    
    M = pm - model
    
    R_values = numpy.einsum ( '...i,...ij,...j->...' , M , numpy.linalg.inv ( covariant_matrices ) , M ) 
        
    return R_values

def compute_log_likelihood ( R_values ):
    log_likelihood = numpy.log ( ( 1. - numpy.exp ( - R_values ** 2 / 2.) ) / ( R_values ** 2 ) ).sum()
    
    return log_likelihood


In [53]:
R = R_values ( data3.proper_motions , data3.proper_motions_err , data3.proper_motions_err_corr , model)

u = compute_log_likelihood (R)

u

-15609.253609503332

# Optimization with Nelder-Mead

In [54]:
from scipy.optimize import minimize

import time

Neval = 1

def callbackF(xk):
    global Neval
    print (Neval)
    print (xk)
    print (metric(xk))
    print ('\n')
    Neval += 1

def metric(x):
    Lmax = 2
    index = 0
    aQlm = {}
    
    for Q in ['E','B']:
        
        aQlm[Q] = {}
        
        for l in numpy.arange(1,Lmax+1):
            
            aQlm[Q][str(l)] = {}
            
            # m=0
            aQlm[Q][str(l)]['0'] = x[index] + (1j) * 0.0
            index += 1
            
            # m>0
            for m in numpy.arange(1,l+1):
                aQlm[Q][str(l)][str(m)] = x[index] + (1j) * x[index+1]
                index += 2
            
            # m<0
            for m in numpy.arange(-l,0):
                aQlm[Q][str(l)][str(m)] = ((-1)**(-m)) * numpy.conj ( aQlm[Q][str(l)][str(-m)] )
                
    vsh_E_coeffs = [ [ aQlm['E'][str(l)][str(m)] for m in numpy.arange(-l,l+1)] for l in numpy.arange(1,Lmax+1)]
    vsh_B_coeffs = [ [ aQlm['B'][str(l)][str(m)] for m in numpy.arange(-l,l+1)] for l in numpy.arange(1,Lmax+1)]
    
    model = generate_model (vsh_E_coeffs , vsh_B_coeffs , data3.positions)
    
    R = R_values ( data3.proper_motions , data3.proper_motions_err , data3.proper_motions_err_corr , model)
    
    u = compute_log_likelihood (R)

    return -u

x0 = numpy.array ( [ random.uniform ( -1e-15 , 1e-15 ) for i in range ( 16 ) ] )
res = minimize(metric, x0, callback=callbackF, method='nelder-mead', options={'xtol': 1.0e-6, 'disp': True})

res.x

1
[-6.41515073e-16  3.37680833e-16  3.14500022e-16 -1.00147338e-15
  8.77206990e-16 -2.28603979e-16 -5.95457744e-16  8.25299791e-16
 -2.82072272e-16  1.03097129e-16  4.86638670e-16 -2.37191815e-16
  5.44803851e-16 -5.09427922e-16 -5.13017691e-16  1.57318665e-16]
15462.592194238174


2
[-6.41515073e-16  3.37680833e-16  3.14500022e-16 -1.00147338e-15
  8.77206990e-16 -2.28603979e-16 -5.95457744e-16  8.25299791e-16
 -2.82072272e-16  1.03097129e-16  4.86638670e-16 -2.37191815e-16
  5.44803851e-16 -5.09427922e-16 -5.13017691e-16  1.57318665e-16]
15462.592194238174


3
[-6.41515073e-16  3.37680833e-16  3.14500022e-16 -1.00147338e-15
  8.77206990e-16 -2.28603979e-16 -5.95457744e-16  8.25299791e-16
 -2.82072272e-16  1.03097129e-16  4.86638670e-16 -2.37191815e-16
  5.44803851e-16 -5.09427922e-16 -5.13017691e-16  1.57318665e-16]
15462.592194238174


4
[-6.41515073e-16  3.37680833e-16  3.14500022e-16 -1.00147338e-15
  8.77206990e-16 -2.28603979e-16 -5.95457744e-16  8.25299791e-16
 -2.82072272e-16

array([-5.49250266e-16,  3.46015898e-16,  3.09127967e-16, -1.04567897e-15,
       -2.52482218e-17, -2.35043658e-16, -5.21217428e-16,  1.02765752e-15,
       -2.56155093e-16,  1.20128261e-16,  5.69728945e-16, -2.83047590e-16,
        5.71439643e-16, -5.22514259e-16, -5.71066211e-16,  2.02417257e-16])

In [ ]:
array([ 2.93466672e-16, -6.08366719e-17,  5.17787746e-16, -2.10567269e-16,
        2.42509365e-17, -1.93634505e-16, -9.27136304e-17,  1.42315527e-17,
       -1.44967058e-16,  5.81291993e-16,  9.00561298e-17,  1.56316100e-16,
       -2.41206913e-16, -7.97276709e-17,  4.11779072e-18,  1.17788137e-16])

In [137]:
def metric(x):
    Lmax = 2
    index = 0
    aQlm = {}
    
    for Q in ['E','B']:
        
        aQlm[Q] = {}
        
        for l in numpy.arange(1,Lmax+1):
            
            aQlm[Q][str(l)] = {}
            
            # m=0
            aQlm[Q][str(l)]['0'] = x[index] + 0*(1j)
            index += 1
            
            # m>0
            for m in numpy.arange(1,l+1):
                aQlm[Q][str(l)][str(m)] = x[index] + (1j)*x[index+1]
                index += 2
            
            # m<0
            for m in numpy.arange(-l,0):
                aQlm[Q][str(l)][str(m)] = ((-1)**l) * numpy.conj(aQlm[Q][str(l)][str(-m)])
                
    vsh_E_coeffs = [ [ aQlm['E'][str(l)][str(m)] for m in numpy.arange(-l,l+1)] for l in numpy.arange(1,Lmax+1)]
    vsh_B_coeffs = [ [ aQlm['B'][str(l)][str(m)] for m in numpy.arange(-l,l+1)] for l in numpy.arange(1,Lmax+1)]
    
    model = generate_model (vsh_E_coeffs , vsh_B_coeffs , data3.positions)
    
    R = R_values ( data3.proper_motions , data3.proper_motions_err , data3.proper_motions_err_corr , model)

    return compute_log_likelihood (R)

metric(numpy.random.uniform(size=16))

    

[[(-0.06877104257936917+0.6624933113393406j),
  (0.9214124775980184+0j),
  (0.06877104257936917+0.6624933113393406j)],
 [(0.11659685837438671-0.7401912966993789j),
  (0.44579986627242596-0.26828942978063675j),
  (0.4994090755475782+0j),
  (0.44579986627242596+0.26828942978063675j),
  (0.11659685837438671+0.7401912966993789j)]]

# Trying to fit with Chris' Python Particle Swarm Optimisation

In [14]:
import PySO

class QSO_VSH_fit(PySO.Model):

    Lmax = 3
    
    names=[]
    for L in numpy.arange(1, Lmax+1):
        for m in numpy.arange(-L, L+1):
            if m==0:
                names += ['Ra^E_{0}_{1}'.format(L,m)]
                names += ['Ra^B_{0}_{1}'.format(L,m)]
            else:
                names += ['Ra^E_{0}_{1}'.format(L,m)]
                names += ['Ra^B_{0}_{1}'.format(L,m)]
                names += ['Ia^E_{0}_{1}'.format(L,m)]
                names += ['Ia^B_{0}_{1}'.format(L,m)]
    
    bounds = [ [-1.0e-14,1.0e-14] for i in range(len(names))]
    

    def log_likelihood(self, param):
        
        vsh_E_coeffs = [ [ 
                            param['Ra^E_{0}_{1}'.format(L,m)] 
                            if m==0 else 
                            param['Ra^E_{0}_{1}'.format(L,m)]+(1j)*param['Ia^E_{0}_{1}'.format(L,m)]
                          for m in numpy.arange(-L, L+1) ] for L in numpy.arange(1, self.Lmax+1)]
        
        vsh_B_coeffs = [ [ 
                            param['Ra^B_{0}_{1}'.format(L,m)] 
                            if m==0 else 
                            param['Ra^B_{0}_{1}'.format(L,m)]+(1j)*param['Ia^B_{0}_{1}'.format(L,m)]
                          for m in numpy.arange(-L, L+1) ] for L in numpy.arange(1, self.Lmax+1)]
        
        print (vsh_E_coeffs)
        
        model = generate_model (vsh_E_coeffs , vsh_B_coeffs , data3.positions)
        R = generate_R ( data3.proper_motions , data3.proper_motions_err , data3.proper_motions_err_corr , model)
        return log_likelhood_fun (R)
    
mymodel = QSO_VSH_fit()

NumParticles = 2

myswarm = PySO.Swarm(mymodel,
                NumParticles,
                Omega = 0.01,
                PhiP = 0.1,
                PhiG = 0.1,
                EnergyTol = 1.0e-8)

myswarm.Run()

[[(-4.051820432552491e-15+3.9019242260068935e-15j), 1.0962514403063393e-15, (-2.1989423583619877e-15+1.5898338470668441e-15j)], [(-4.005078512411002e-15+8.295448331349153e-15j), (7.242640554933391e-15-7.022912072523901e-15j), 7.407801139573162e-16, (-5.394325112551346e-15+9.140480362437914e-15j), (-3.866305737185125e-15-1.4372516862603125e-15j)], [(9.905129645633751e-15-1.2045374537853373e-15j), (-4.591105300268807e-15+1.6206152525940413e-16j), (-1.4651795083578574e-15-2.3169299929584544e-15j), 2.00678369665432e-15, (8.961771647529057e-15-3.435506878025172e-15j), (-2.777910391239437e-15-6.516987950664284e-15j), (-7.6636705852496e-15+1.4687411361732721e-15j)]]
[[(-4.86130129408709e-15+9.746539081626012e-15j), 1.7986352892125863e-15, (7.270009629945627e-15-2.0299349018935748e-15j)], [(-3.490502032823688e-15-1.715352162705143e-15j), (-4.862114468593031e-15-5.833711376264423e-15j), -8.561320209723371e-15, (4.63599678108349e-15+5.5491893756491685e-15j), (-8.754921473469426e-15+7.54194650712

KeyboardInterrupt: 

In [ ]:
data3.proper_motions_err_corr

def tangent_err_to_Cartesrian (positions , pms , pm_errs , pm_err_corr):
    for i , point in enumerate (positions):
        ra = point[0]
        dec = point[1]
        
        pmra_err = pm_errs[i,0]
        pmdec_err = pm_errs[i,1]
        
        pmra_pmdec_err_corr = pm_err_corr[i,0]
        
        R = numpy.array ( [ [ - numpy.cos ( dec ) * numpy.sin ( ra ) , - numpy.sin ( dec ) * numpy.cos ( ra ) ] ,
                            [ numpy.cos ( dec) * numpy.sin ( ra ) , - numpy.sin ( dec ) * numpy.sin ( ra ) ] ,
                            [ - numpy.cos ( dec ) , 0 ] ] )
        
        pm_err_matrix = numpy.array ( [ [ pmra_err , numpy.sqrt ( numpy.abs (pmra_err * pmdec_err * pmra_pmdec_err_corr ) ) ] ,
                                        [ numpy.sqrt ( numpy.abs (pmra_err * pmdec_err * pmra_pmdec_err_corr ) ) , pmdec_err ] ] )
        
        pm_err_matrix_Cartesian = R.dot(pm_err_matrix).dot(R.transpose())
        
#         print (numpy.linalg.eig(pm_err_matrix_Cartesian))
    
tangent_err_to_Cartesrian (data3.positions , data3.proper_motions , data3.proper_motions_err , data3.proper_motions_err_corr)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

def plot_pm_err_ellipse (pm , pm_err , pm_err_corr):
    i = 575
    
    covariance_matrix = numpy.array ( [ [ pm_err[i,0] * pm_err[i,0] , pm_err_corr[i] * pm_err[i,0] * pm_err[i,1] ],
                                        [ pm_err_corr[i] * pm_err[i,1] * pm_err[i,0] , pm_err[i,1] * pm_err[i,1] ] ] )
    
    eigenvalues , eigenvectors = numpy.linalg.eig ( covariance_matrix )
    
    box_size = numpy.max ( [ numpy.sqrt(eigenvalues[0]) , numpy.sqrt(eigenvalues[1]) , numpy.linalg.norm ( pm[i] ) ] )
    
    plt.figure(figsize=(10,10))
    plt.axes().set_aspect('equal')
    ax = plt.gca()
    
    plt.xlim ( [ pm[i,0] - 1.5 * box_size , pm[i,0] + 1.5 * box_size ] )
    plt.ylim ( [ pm[i,1] - 1.5 * box_size , pm[i,1] + 1.5 * box_size ] )
    
    plt.arrow(0 , 0 , pm[i,0] , pm[i,1] , width=box_size * 1e-3 , head_width=box_size * 0.5e-1, head_length=box_size * 1e-1 , color='k')
            
    plt.arrow(pm[i,0] , pm[i,1] , numpy.sqrt(eigenvalues[0]) * eigenvectors[0,0] , numpy.sqrt(eigenvalues[0]) * eigenvectors[0,1] , width=box_size * 1e-3 , head_width=box_size * 0.5e-1, head_length=box_size * 1e-1 , color='orange')
    
    plt.arrow(pm[i,0] , pm[i,1] , numpy.sqrt(eigenvalues[1]) * eigenvectors[1,0] , numpy.sqrt(eigenvalues[1]) * eigenvectors[1,1] , width=box_size * 1e-3 , head_width=box_size * 0.5e-1, head_length=box_size * 1e-1 , color='y')
    
    angle = - numpy.rad2deg ( numpy.arctan2 ( eigenvectors[1,0] , eigenvectors[1,1] ) )
            
    ell = Ellipse(xy=(pm[i,0] , pm[i,1]), width=2 * numpy.sqrt(eigenvalues[0]), height=2 * numpy.sqrt(eigenvalues[1]), angle = angle)
    
    ax.add_patch(ell)
    ax.set_aspect('equal')
            
    plt.show()
    
    return covariance_matrix

print (plot_pm_err_ellipse ( data3.proper_motions , data3.proper_motions_err , data3.proper_motions_err_corr ))

In [ ]:
initial_pos = type3_sample
final_pos = type3_sample + type3_sample_pm

lines_to_plot = []

for i,_ in enumerate (initial_pos):
    longs = numpy.linspace(initial_pos[i,0], final_pos[i,0], 10)
    lats = numpy.linspace(initial_pos[i,1], final_pos[i,1], 10)
    
    line_m = GeographicToMollweide ( numpy.array ( [longs , lats] ).transpose() )
    
    lines_to_plot.append( line_m.transpose() )


In [ ]:
initial_pos_m = GeographicToMollweide ( initial_pos )
final_pos_m = GeographicToMollweide ( final_pos )
diff_m = final_pos_m - initial_pos_m

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))

t = numpy.linspace (0, 2 * numpy.pi, 100)
plt.plot ( 2 * numpy.sqrt(2) * numpy.cos(t) , numpy.sqrt(2) * numpy.sin(t) , linewidth=0.5 )

for i,_ in enumerate(initial_pos_m):
    plt.arrow( initial_pos_m[i,0] , initial_pos_m[i,1] , diff_m[i,0] , diff_m[i,1] , head_width=0.025, head_length=0.05)

plt.show()

In [ ]:
import spherepy as sp
c = sp.random_coefs(4, 4) # generate some random coefficients
print (type(c))
# sp.pretty_coefs(c)
# p = sp.ispht(c, 50, 50) # inverse spherical transform to pattern
# sp.plot_sphere_mag(p) # plot the magnitude of the pattern

In [ ]:
diff_m.argmin(axis=0)

In [ ]:
# look up units